In [ ]:
from __future__ import absolute_import
from requests.auth import HTTPBasicAuth
from collections import defaultdict
import requests
import json
import imp
import django.utils.six
import getpass
import pprint

# default is volatile memory backend, but also supports redis and postgres backends
# the user ID that updates any permissions is always added to the write permission list

# run as admin, loop through all projects and create bucket for each project
# update permissions to match project contributors' permissions

email = raw_input('Enter your email: ')
password = getpass.getpass('Enter your password: ')

headers = {'Content-Type': 'application/json'}
auth = (email, password)

r = requests.get('http://0.0.0.0:8888/v1/',auth=HTTPBasicAuth('user', 'pass')).json()

data = requests.get('http://localhost:8000/v2/nodes?format=json', auth=auth).json()
if 'errors' in data:
    print('Invalid email/password')

counter = 0; 
try:
    for item in data['data']:
        counter += 1;
except:
    KeyError
    
# loop through projects 
input = {}     
for i in range(0,counter):
    data = requests.get('http://localhost:8000/v2/nodes?format=json', auth=auth).json()
    input = {}
    input['title'] = data['data'][i]['attributes']['title']
    
    # create a metadata bucket for each project 
    r = requests.post('http://0.0.0.0:8888/v1/buckets',json={'data':{'id': input['title']}},auth=HTTPBasicAuth('user', 'pass'),headers=headers).json()
    title = input['title']
    
    # check if a project is public 
    # if public then grant read permissions to all authenticated users
    pub = ''
    input = {}
    input['public'] = data['data'][i]['attributes']['public']
    if input['public'] == True:
        pub = 'system.Authenticated'
    
    pid = data['data'][i]['id'] # project id
    data = requests.get('http://localhost:8000/v2/nodes/'+pid+'/contributors/?format=json', auth=auth).json()
    
    contrib = 0; 
    for item in data['data']:
        contrib += 1;
    
    per = defaultdict(list)
    
    input = {}
    for j in range(0,contrib):
        input['permissions'] = data['data'][j]['attributes']['permission']
        cname = data['data'][j]['embeds']['users']['data']['attributes']['full_name'] # contributor name
        
        # check contributors' permissions and update them in project bucket
        # (write implies read, write, and delete in Kinto)
        if 'admin' in input['permissions']:
            per['collection:create'].append(cname)
            per['group:create'].append(cname)
            per['write'].append(cname)
        elif ('write' in input['permissions']):
            per['write'].append(cname)
        elif 'read' in input['permissions']:
            per['read'].append(cname)
    if pub != '':
        per['read'].append(pub)
    r = requests.patch('http://0.0.0.0:8888/v1/buckets/'+title,json={'permissions': per},auth=HTTPBasicAuth('user', 'pass'), headers=headers).json()
